In [4]:
import pandas as pd
import krippendorff as k
import numpy as np
import collections
import gensim
from gensim import corpora, models
from string import punctuation
import import_ipynb
import SerbStemmer
import nltk
from nltk.tokenize import RegexpTokenizer
nltk.download('punkt')
from SerbStemmer import stem_str
tokenizer = RegexpTokenizer(r'\w+')
import transliterate
from transliterate import translit

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\LenovoPC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NER and Sentiment was obtaioned via the transliterate package which can only run on a non-Windows OS. The code for this part of the analysis was run in a virtual machine, but is presented here. Do not run the following cell.

In [ ]:
from polyglot.text import Text, Word
import transliterate
from transliterate import translit

## Create stopword list

Extracting stopwords from all data to create specialized stopwords list. Read in all of our data, use both titles and texts. Place all in flat list with lowercase. Get top 200 and clean up manually removing words such as "Russia", "Serbia", "Trump", "Vucic".

Also take stopwords from https://github.com/Xangis/extra-stopwords/blob/master/serbian. Clean up both lists and drop duplicates. This is our stopword list. 

In [ ]:
vostok = pd.read_csv('vostok_clean.csv', sep = ',')
sputnik = pd.read_csv('sputnik_clean.csv', sep = ',')
n1 = pd.read_csv('n1_clean.csv', sep = ',')
voa = pd.read_csv('voa_clean.csv', sep = ',')
rfe = pd.read_csv('rfe_clean.csv', sep = ',')
all_data = pd.concat([vostok,sputnik, n1, voa, rfe], axis=0, ignore_index=True)

In [39]:
titles = all_data['title'].tolist()
text = all_data['text'].tolist()
all_text = titles+text
flat_list = []
for sublist in all_text:
    for word in sublist.split():
        flat_list.append(word.lower())

In [41]:
counter = collections.Counter(flat_list)
stopword_filter = []
for word in counter.most_common(200):
    stopword_filter.append(word)
stopword_filter = pd.DataFrame(stopword_filter)

In [42]:
git_list = pd.read_csv("Git_stop_list.csv")
git_list = pd.DataFrame(git_list)
git_list['ba?;'] = git_list['ba?;'].str.replace('?', 'ć').str.replace(';', '')
git_list['ba?;'] = git_list['ba?;'].astype(str)
git_list.columns = ['words']

In [43]:
stopword_filter = pd.DataFrame(stopword_filter[0])
stopword_filter.columns = ['words']
all_stop = stopword_filter.append(git_list, ignore_index = True)
all_stop = all_stop.drop_duplicates()
all_stop = all_stop.reset_index(drop = True)

In [44]:
all_stop['words'][206] = all_stop['words'][206].replace('doćao', 'došao')
all_stop['words'][226] = all_stop['words'][226].replace('joć', 'još')
all_stop['words'][241] = all_stop['words'][241].replace('nećto', 'nešto')
all_stop['words'][245] = all_stop['words'][245].replace('nićta', 'ništa')
all_stop['words'][255] = all_stop['words'][255].replace('otićao', 'otišao')
all_stop['words'][266] = all_stop['words'][266].replace('ćta', 'šta')
all_stop['words'][283] = all_stop['words'][283].replace('vać', 'vaš')
all_stop['words'][287] = all_stop['words'][287].replace('viće', 'više')

In [ ]:
#only keep relevant words from list
all_stop = all_stop.drop([14, 27, 35, 47, 49, 62, 77, 79, 86, 92, 93,94, 96, 103, 105, 120, 125, 127, 130, 132, 133, 134, 137, 140, 141, 154, 155, 158, 160, 165, 166, 171, 172, 173, 178, 181, 190,196,198, 289, 290, 291])
all_stop.to_csv('stopwords.csv', encoding = 'utf-8', index = False )
stopwords = pd.read_csv('stopwords.csv')
stopwords = stopwords['words'].tolist()

In [21]:
coder_1 = pd.read_csv('coder_1_data.csv', sep = ',')
coder_4 = pd.read_csv("coder_4_data.csv", sep =',')
coder_3 = pd.read_csv('coder_3_data.csv', sep = ',')
coder_5 = pd.read_csv('coder_5_data.csv', sep = ',')

Here we are extracting the automated features from all of the texts. These are the following: 
Headline Length 
Article Length 
Average Word Length 
Language complexity (proportion of unique words to all words)
Ratio of stopwords to all words 
Named Entity recognition (title and text) 
Top five words after stopwords (basic topic modelling) 

In [160]:
def automated_features(name):
    name['text'] = name['text'].str.replace(',', '').str.replace('<br/>', '')
    s = stopwords
    name['article_len'] = name['text'].str.split().apply(len)#  length of text (no. of words)
    name['string_len'] = name['text'].apply(len)#length of total string (characters)
    name['stop_words'] = [list(set(x).intersection(s)) for x in name['text'].str.split()] # stopwords
    name['stop_words_t'] = [list(set(x).intersection(s)) for x in name['title'].str.split()] # stopwords
    name['title_len'] = name['title'].str.split().apply(len) # no of words in title = title length
    name['no_stop_words'] = name['stop_words'].str.len()# number of stopwords number
    name['ratio_not_stop_w'] = name['no_stop_words']/name['article_len']
    name['title_len'] = name['title'].str.split().apply(len)
    name['no_stop_words_t'] = name['stop_words_t'].str.len()# number of stopwords number
    name['ratio_not_stop_w_t'] = name['no_stop_words_t']/name['title_len']
    name['avg_word_len'] = name['string_len']/name['article_len'] #derived avg. word length
    name['wordscounter'] = name['text'].str.lower().str.split().apply(set).apply(len) #no of unique words
    name['ratio_unique_words'] = name['wordscounter']/name['article_len'] #prop of unique word to total words = language complexity
    del name['wordscounter']
    del name['string_len']
    del name['no_stop_words']
    del name['stop_words']
    del name['stop_words_t']
    del name['no_stop_words_t']
    return name
  

def ner(name):
    tekstovi = name['text'].tolist()
    tekstovi = [translit(tekst, 'sr', reversed = False) for tekst in tekstovi]
    tekstovi = [Text(tekst, hint_language_code = 'sr') for tekst in tekstovi]
    tekstovi = [tekst.entities for tekst in tekstovi]
    titlovi = name['title'].tolist()
    titlovi = [translit(titl, 'sr', reversed = False) for titl in titlovi]
    titlovi = [Text(titl, hint_language_code = 'sr') for titl in titlovi]
    titlovi = [titl.entities for titl in titlovi]
    all_ner = list(zip(titlovi, tekstovi)) 
    ner = []
    for line in all_ner:
        inner_ner = []
        for lines in line:
            for element in lines:
                for el in element:
                    inner_ner.append(el)
        ner.append(inner_ner)
    ner = [list(set(line)) for line in ner] #return only unique values 
    ner = [[translit(x, 'sr', reversed = True) for x in ner_value] for ner_value in ner]
    ner = [[x.lower() for x in ner_value] for ner_value in ner]
    return ner

## Do not run the NER and Sen functions

In [22]:
coder_1_features = automated_features(coder_1)
coder_3_features = automated_features(coder_3)
coder_4_features = automated_features(coder_4)
coder_5_features = automated_features(coder_5)

coder_4_features['ner'] = pd.DataFrame({'ner': coder_4_ner})
coder_5_features['ner'] = pd.DataFrame({'ner': coder_5_ner})
coder_3_features['ner'] = pd.DataFrame({'ner': coder_3_ner})
coder_1_features['ner'] = pd.DataFrame({'ner': coder_1_ner})
coder_4_features['sen'] = pd.DataFrame({'sen': coder_4_sen})
coder_5_features['sen'] = pd.DataFrame({'sen' : coder_5_sen})
coder_1_features['sen'] = pd.DataFrame({'sen': coder_1_sen})   
coder_3_features['sen'] = pd.DataFrame({'sen': coder_3_sen})
coder_1_features['ner'] = coder_1_features['ner'].str.lower()
coder_5_features['ner'] = coder_5_features['ner'].str.lower()
coder_4_features['ner'] = coder_4_features['ner'].str.lower()
coder_3_features['ner'] = coder_3_features['ner'].str.lower()
#these datasets make up coders.csv

In [11]:
#creating the training set with all features from coders and the automated feature extraction
#coders data set is available in the repository
train_pd= pd.read_csv("coders.csv", encoding = 'utf-8')
stopwords = pd.read_csv("stopwords.csv")
def connect_frames(name, tag):
    name = pd.read_csv('{}_features.csv'.format(name), sep = ',', encoding = 'utf-8')
    frames = train_pd
    frames = frames.loc[(frames['Q3'] == 'Machine set') & (frames['Q1'] == '{}'.format(tag))]
    frames = frames[['Q2','A', 'B', 'C', 'D', 'E', 'F', 'G']]
    frames['Q2'] = frames['Q2'].apply(int)
    name['Q2'] = name.index
    new = pd.merge(name,frames, how = 'left', on = 'Q2')
    return new
#create all feature data set from all 
coder_1 = connect_frames('coder_1', 'cd_1')
coder_3 = connect_frames('coder_3', 'cd_3')
coder_4 = connect_frames('coder_4', 'cd_4')
coder_5 = connect_frames('coder_5', 'cd_5')
train = pd.concat([coder_1,coder_3, coder_4, coder_5], axis=0, ignore_index=True)
train = train[['text', 'country-source', 'Q2', 'D', 'E', 'F', 'G', 'article_len', 'ratio_not_stop_w', 'avg_word_len', 'title_len', 'ratio_unique_words', 'ner']]

In [145]:
len(train)

1108

The same functions are applied to the entire data set remaining.

In [7]:
test_data = pd.read_csv('test_data.csv')
test_data = test_data[test_data.text.apply(lambda x: isinstance(x,str))]
test = automated_features(test_data)
test_ner = ner(test)
test['ner'] = pd.DataFrame({'ner' : test_ner})
test.to_csv('test_features.csv')

#  Krippendorf's alpha

In [13]:
coders = pd.read_csv('coders.csv')
tags = ['cd_1', 'cd_3', 'cd_4', 'cd_5']
coders = coders.drop(1713)#remove 
coders = coders.drop(1740)#remove
coders = coders.loc[coders['Q3'] == 'Rel2']
def krip_alpha(name, tag, field):
    name = coders[coders['Q1'] == '{}'.format(tag)]
    name = pd.DataFrame(name['{}'.format(field)])
    name['{}'.format(field)] = np.where(name['{}'.format(field)]== 'Yes', 1, 0)
    name_list = name['{}'.format(field)].tolist()
    return name_list
coders = pd.read_csv('coders.csv')
coders = coders.loc[coders['Q3'] == 'Machine set']
len(coders)

In [11]:
k_values = []
for tag in tags:
    coder = krip_alpha('name', tag, 'G')
    k_values.append(coder)
k.alpha(k_values)

1283